# Import Dataset from MSSQL

In [ ]:
pwd

In [ ]:
import os
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
output_dir = pathlib.Path('{}/output/'.format(current_dir))
pathlib.Path.mkdir(output_dir, mode=0o777, parents=True, exist_ok=True)

# Only IO 집단 생성을 위한 SQL query문. 필요한 쿼리문 택1

### Atlas 에서 코호트 생성 안했을 경우 (SQL query)
#### 1) mssql      ; query_population_sql(mssql).sql
#### 2) postgres   ; query_population_sql(postgres).sql
### Atlas에서 코호트 생성했을 경우 (SQL query)
#### 1) mssql      ; query_cohort_from_atlas(mssql).sql
#### 2) postgres   ; query_cohort_from_atlas(postgres).sql

### 이하 python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import os
import pandas as pd
import numpy as np
#!pip install pymssql
import pymssql
from datetime import timedelta

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

# connection DataBase (해당 DB의 명령문 사용)

### Postgresql (계정유출 주의)

In [ ]:
# import pandas.io.sql as psql
# import psycogp2 as pg
# conn = pg.connect(host='111.1.11.11', dbname='cdm', user='1111', password='1111', port='5432')
# print('connection')

# sql = 'select * from cdm.cdm.person'
# population = psql.read(sql=sql1, con=conn)
# population.to_csv('population.txt', index=False)

### Mssql (계정유출 주의)

In [ ]:
import pymssql
conn= pymssql.connect(server="111.111.111.111", user="user", password="password", port=22, database='db', as_dict=False)
print("connection")

### 용량이 클 경우, population 조건 걸어서 일부 데이터만 불러오기

In [ ]:
# 용량이 클 경우, population 조건 걸어서 일부 데이터만 불러오기
# 기관별 이름 설정 
db_name='CDM'
schema_name = 'dbo'
population = 'population_only_IO'
measurement = 'measurement'
drug = 'drug_exposure'
procedure = 'procedure_occurrence'
condition = 'condition_occurrence'
concept = 'concept'

sqlq0 = 'select * from CDM.dbo.population_only_IO'
population_df = pd.read_sql(sql=sqlq0, con=conn)

sqlq1 ="select person_id, measurement_concept_id, measurement_date, value_as_number from {}.{}.[{}] ".format(db_name,schema_name,measurement) +\
    "where measurement_concept_id!=0 and value_as_number is not null and person_id in (select distinct person_id from {}.{}.[{}])".format(db_name,schema_name,population)
measurement_df = pd.read_sql(sql=sqlq1, con=conn)

sqlq2="select person_id, drug_concept_id, drug_exposure_start_date, quantity from {}.{}.[{}] ".format(db_name,schema_name,drug) +\
    "where drug_concept_id!=0 and quantity is not null and person_id in (select distinct person_id from {}.{}.[{}])".format(db_name,schema_name,population)
drug_df = pd.read_sql(sql=sqlq2, con=conn)

sqlq3="select person_id, procedure_concept_id, procedure_date from {}.{}.[{}] ".format(db_name,schema_name,procedure) +\
    "where procedure_concept_id!=0 and person_id in (select distinct person_id from {}.{}.[{}] )".format(db_name,schema_name,population)
procedure_df = pd.read_sql(sql=sqlq3, con=conn)

sqlq4="select person_id, condition_concept_id, condition_start_date from {}.{}.[{}] ".format(db_name,schema_name,condition) +\
    "where condition_concept_id!=0 and person_id in (select distinct person_id from {}.{}.[{}] )".format(db_name,schema_name,population)
condition_df = pd.read_sql(sql=sqlq4, con=conn)

sqlq5="select distinct concept_id, concept_name from {}.{}.[{}]".format(db_name, schema_name, concept)
concept_df = pd.read_sql(sql=sqlq5, con=conn)

conn.close()

In [ ]:
population_df

### or

### 전체 데이터 불러오기(셧다운 가능성 높아서 권장하지 않음)

In [ ]:
# #전체 데이터 불러오기(셧다운 가능성 높아서 권장하지 않음)

# # 기관별 이름 설정 
# db_name='CDM'
# schema_name = 'dbo'
# population = 'population_only_IO'
# measurement = 'measurement'
# drug = 'drug_exposure'
# procedure = 'procedure_occurrence'
# condition = 'condition_occurrence'
# concept = 'concept'

# sqlq1="select * from {}.{}.[{}]".format(db_name,schema_name,population)
# population = pd.read_sql(sql=sqlq1, con=conn)


# sqlq1="select top(10) * from {}.{}.[{}] where measurement_concept_id!=0 and value_as_number is not null".format(db_name,schema_name,measurement)
# measurement = pd.read_sql(sql=sqlq1, con=conn)


# sqlq2="select top(10)  * from {}.{}.[{}] where drug_concept_id!=0 and quantity is not null".format(db_name,schema_name,drug)
# drug = pd.read_sql(sql=sqlq2, con=conn)

# sqlq3="select top(10)  * from {}.{}.[{}] where procedure_concept_id!=0".format(db_name,schema_name,procedure)
# procedure = pd.read_sql(sql=sqlq3, con=conn)

# sqlq4="select top(10)  * from {}.{}.[{}] where condition_concept_id!=0".format(db_name,schema_name,condition)
# condition = pd.read_sql(sql=sqlq4, con=conn)


# sqlq5="select distinct concept_id, concept_name from CDM.dbo.[concept]".format(db_name,schema_name,concept)
# concept = pd.read_sql(sql=sqlq5, con=conn)

# conn.close()

# Save dataset

In [ ]:
pwd

In [ ]:
population_df.to_csv('{}/population.txt'.format(output_dir),index=False)
measurement_df.to_csv('{}/measurement.txt'.format(output_dir),index=False)
drug_df.to_csv('{}/drug.txt'.format(output_dir),index=False)
procedure_df.to_csv('{}/procedure.txt'.format(output_dir),index=False)
condition_df.to_csv('{}/condition.txt'.format(output_dir),index=False)
concept_df.to_csv('{}/concept.txt'.format(output_dir),index=False)

# Load dataset

In [ ]:
pwd

In [ ]:
# 데이터 불러오기
population_df=pd.read_csv('{}/population.txt'.format(output_dir))
measurement_df=pd.read_csv('{}/measurement.txt'.format(output_dir))
drug_df=pd.read_csv('{}/drug.txt'.format(output_dir))
procedure_df=pd.read_csv('{}/procedure.txt'.format(output_dir))
condition_df=pd.read_csv('{}/condition.txt'.format(output_dir))
concept_df=pd.read_csv('{}/concept.txt'.format(output_dir))

In [ ]:
measurement_df = measurement_df[["person_id","measurement_concept_id","measurement_date","value_as_number"]]
drug_df = drug_df[["person_id","drug_concept_id","drug_exposure_start_date","quantity"]]
procedure_df = procedure_df[["person_id","procedure_concept_id","procedure_date"]]
condition_df = condition_df[["person_id","condition_concept_id","condition_start_date"]]
concept_df = concept_df[["concept_id","concept_name"]]

In [ ]:
# # 그룹바이 해서 10건 미만 리스트업
# c=measurement_df.columns
# gb=measurement_df[c].groupby(["measurement_concept_id"]).size().reset_index(name="cnt")
# gb_10=gb[gb["cnt"]<10]
# gb_df=pd.DataFrame(gb_10["measurement_concept_id"].unique())
# gb_df["measurement_concept_id"]=gb_df[0]
# # 해당 measurement_concept_id 를 포함하는 레코드 찾기
# idx=[]
# for i in gb_df["measurement_concept_id"]:    
#     d= measurement_df[measurement_df["measurement_concept_id"]==i]
#     d1=[i for i in d.index]    
#     idx.extend(d1)  
# # 인덱스로 제거 
# measurement_df2=measurement_df.drop(idx)
# measurement_df=measurement_df2.reset_index(drop=True)

# Only IO: generate each domain dataset

In [ ]:
# population for each domains
population_df=population_df[["subject_id","cohort_start_date","age","gender_source_value","person_id"]]
population_df=population_df.drop_duplicates()
len(population_df)
## measurement
IO_measurement = pd.merge(population_df, measurement_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_measurement = IO_measurement.query('cohort_start_date<=measurement_date')
IO_measurement = pd.merge(IO_measurement, concept_df, left_on=["measurement_concept_id"],right_on=["concept_id"],how="inner")
IO_measurement = IO_measurement.reset_index(drop=True)
#drug
IO_drug = pd.merge(population_df, drug_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_drug = IO_drug.query('cohort_start_date<=drug_exposure_start_date')
IO_drug = pd.merge(IO_drug,concept_df,left_on=["drug_concept_id"],right_on=["concept_id"],how="inner")
IO_drug = IO_drug.reset_index(drop=True)
#procedure
IO_procedure = pd.merge(population_df, procedure_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_procedure = IO_procedure.query('cohort_start_date<=procedure_date')
IO_procedure = pd.merge(IO_procedure,concept_df,left_on=["procedure_concept_id"],right_on=["concept_id"],how="inner")
IO_procedure = IO_procedure.reset_index(drop=True)
#condition
IO_condition = pd.merge(population_df, condition_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_condition = IO_condition.query('cohort_start_date<=condition_start_date')
IO_condition = pd.merge(IO_condition,concept_df,left_on=["condition_concept_id"],right_on=["concept_id"],how="inner")
IO_condition = IO_condition.reset_index(drop=True)

In [ ]:

output_feature_dir = pathlib.Path('{}/output_feature/'.format(current_dir))
meas_concept_df=pd.read_csv('{}/meas_concept_similarity.txt'.format(output_feature_dir), index_col=False)
drug_concept_df=pd.read_csv('{}/drug_concept_similarity.txt'.format(output_feature_dir), index_col=False)
proc_concept_df=pd.read_csv('{}/proc_concept_similarity.txt'.format(output_feature_dir), index_col=False)
cond_concept_df=pd.read_csv('{}/cond_concept_similarity.txt'.format(output_feature_dir), index_col=False)
print(len(meas_concept_df), len(drug_concept_df), len(proc_concept_df), len(cond_concept_df))

In [ ]:
def createBackupColumn(df):
    df['concept_id_org'] = df['concept_id'] 
    df['concept_name_org'] = df['concept_name'] 
    return df

IO_measurement = createBackupColumn(IO_measurement)
IO_drug = createBackupColumn(IO_drug)
IO_procedure = createBackupColumn(IO_procedure)
IO_condition = createBackupColumn(IO_condition)

In [ ]:
def convertToCommonConceptID(concept_df, domain_df):
    for index, [cid_1, cname_1, cid_2, cname_2] in concept_df.iterrows():
        #print(len(domain_df[domain_df['concept_id_org']==cid_2]), len(domain_df[domain_df['concept_id_org']==cid_1]))
        domain_df['concept_id'] = domain_df['concept_id_org'].replace(cid_2, cid_1)
        domain_df['concept_name'] = domain_df['concept_name_org'].replace(cname_2, cname_1)
    return domain_df 

IO_measurement = convertToCommonConceptID(meas_concept_df, IO_measurement)
print(len(IO_measurement[IO_measurement['concept_id'] != IO_measurement['concept_id_org']]))

IO_drug = convertToCommonConceptID(drug_concept_df, IO_drug)
print(len(IO_drug[IO_drug['concept_id'] != IO_drug['concept_id_org']]))

IO_procedure = convertToCommonConceptID(proc_concept_df, IO_procedure)
print(len(IO_procedure[IO_procedure['concept_id'] != IO_procedure['concept_id_org']]))

IO_condition = convertToCommonConceptID(cond_concept_df, IO_condition)
print(len(IO_condition[IO_condition['concept_id'] != IO_condition['concept_id_org']]))


In [ ]:
del meas_concept_df
del drug_concept_df
del proc_concept_df
del cond_concept_df

del population_df
del measurement_df
del drug_df
del procedure_df
del condition_df
del concept_df

# CTCAE outcomes: generate each outcome dataset

# TARGETTING

In [ ]:
def measurement_targetting(abrnoaml_ctcae, IO_measurement):
    abnormal_ctcae_idx = abrnoaml_ctcae.sort_values(by=["person_id","measurement_date"],ascending=True)\
    .drop_duplicates(subset="person_id",keep="first").index
    
    abrnoaml_ctcae["first_abnormal_date"] = abrnoaml_ctcae.loc[abnormal_ctcae_idx]["measurement_date"]

    # target 의 person_id
    target_person=abrnoaml_ctcae["person_id"].unique()
    first_abnormal_date=abrnoaml_ctcae[abrnoaml_ctcae["first_abnormal_date"].notnull()].index
    #print("abnormal person:",len(target_person))   

    # abnormal 환자의 모든 레코드를 담기위해
    # abnormal person_id 수집
    abnormal_idx=[]
    for i in target_person:        
        abnormal_idx.extend(IO_measurement[IO_measurement["person_id"]==i].index)

    #abnormal dataframe
    abnormal_df=IO_measurement.loc[abnormal_idx]  
    abnormal_df["first_abnormal_date"]=abnormal_df.loc[first_abnormal_date]["measurement_date"]

    #해당 person 의 first_abnormal_date 채우기(그래야 row 별로 비교 가능)
    for person_id, group_data in abnormal_df.groupby(["person_id"]):    
        all_date=group_data.first_abnormal_date.unique()
        first_abnormal_date = [i for i in all_date][1]
        #print(first_abnormal_date)
        abnormal_df["first_abnormal_date"][abnormal_df["person_id"]==person_id]=first_abnormal_date

    # 전체 index 에서 abnormal index 제거해서 normal index 찾기
    normal_idx=IO_measurement.index.difference(abnormal_idx)
    normal_idx=normal_idx.tolist()
    normal_df=IO_measurement.loc[normal_idx] 

    # label 1,0 채우기
    abnormal_df["label"]=1
    normal_df["label"]=0
    print(len(abnormal_df["person_id"].unique())) # 
    print(len(normal_df["person_id"].unique()))   # 

    # reset_index
    abnormal_df=abnormal_df.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)
    normal_df=normal_df.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)

    # imputation
    abnormal_df["first_abnormal_date"]=abnormal_df["first_abnormal_date"].fillna('1970-01-01')
    abnormal_df["person_id"]=abnormal_df["person_id"].fillna(abnormal_df["subject_id"])
    abnormal_df["measurement_date"]=abnormal_df["measurement_date"].fillna(abnormal_df["cohort_start_date"])
    normal_df["person_id"]=normal_df["person_id"].fillna(normal_df["subject_id"])
    normal_df["measurement_date"]=normal_df["measurement_date"].fillna(normal_df["cohort_start_date"])

    abnormal_df=abnormal_df.fillna('0')
    normal_df=normal_df.fillna('0')
    
    dataset = pd.concat([abnormal_df,normal_df])
    dataset.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)
    
    return dataset

# medical history wash out and abnormal targetting

# Lab HOI

In [ ]:
# 1. value 1개짜리 & decreased
Hypoalbuminemia_G1=[3024561,'<',3.3]
Hypoalbuminemia_G2=[3024561,'<',3]
Hypoalbuminemia_G3=[3024561,'<',2]
Hypokalemia_G1=[3023103,'<',3.5]
Hypokalemia_G3=[3023103,'<',3]
Hypokalemia_G4=[3023103,'<',2.5]
Fibrinogen_decreased_G1=[3016407,'<',200]
Fibrinogen_decreased_G2=[3016407,'<',150]
Fibrinogen_decreased_G3=[3016407,'<',100]
Fibrinogen_decreased_G4=[3016407,'<',50]
Hypoglycemia_G1=[3004501,'<',70]
Hypoglycemia_G2=[3004501,'<',55]
Hypoglycemia_G3=[3004501,'<',40]
Hypoglycemia_G4=[3004501,'<',30]
Hypomagnesemia_G1=[3012095,'<',0.75]
Hypomagnesemia_G2=[3012095,'<',0.5]
Hypomagnesemia_G3=[3012095,'<',0.4]
Hypomagnesemia_G4=[3012095,'<',0.3]
Hyponatremia_G1=[3019550,'<',135]
Hyponatremia_G2=[3019550,'<',130]
Hyponatremia_G3=[3019550,'<',125]
Hyponatremia_G4=[3019550,'<',120]
Lymphocyte_count_decreased_G1=[3019198,'<',0.9]
Lymphocyte_count_decreased_G2=[3019198,'<',0.8]
Lymphocyte_count_decreased_G3=[3019198,'<',0.5]
Lymphocyte_count_decreased_G4=[3019198,'<',0.2]
Acidosis_arterial_G1=[3019977,'<', 7.35]
Acidosis_arterial_G3=[3019977,'<', 7.3]

# value 1개 짜리
    #decreased

abnormals=[
    Hypoalbuminemia_G1
    ,Hypoalbuminemia_G2
    ,Hypoalbuminemia_G3
    ,Hypokalemia_G1
    ,Hypokalemia_G3
    ,Hypokalemia_G4
    ,Fibrinogen_decreased_G1
    ,Fibrinogen_decreased_G2
    ,Fibrinogen_decreased_G3
    ,Fibrinogen_decreased_G4
    ,Hypoglycemia_G1
    ,Hypoglycemia_G2
    ,Hypoglycemia_G3
    ,Hypoglycemia_G4
    ,Hypomagnesemia_G1
    ,Hypomagnesemia_G2
    ,Hypomagnesemia_G3
    ,Hypomagnesemia_G4
    ,Hyponatremia_G1
    ,Hyponatremia_G2
    ,Hyponatremia_G3
    ,Hyponatremia_G4
    ,Lymphocyte_count_decreased_G1
    ,Lymphocyte_count_decreased_G2
    ,Lymphocyte_count_decreased_G3
    ,Lymphocyte_count_decreased_G4
    ,Acidosis_arterial_G1
    ,Acidosis_arterial_G3]

abnormals_nm=[
    'Hypoalbuminemia_G1'
    ,'Hypoalbuminemia_G2'
    ,'Hypoalbuminemia_G3'
    ,'Hypokalemia_G1'
    ,'Hypokalemia_G3'
    ,'Hypokalemia_G4'
    ,'Fibrinogen_decreased_G1'
    ,'Fibrinogen_decreased_G2'
    ,'Fibrinogen_decreased_G3'
    ,'Fibrinogen_decreased_G4'
    ,'Hypoglycemia_G1'
    ,'Hypoglycemia_G2'
    ,'Hypoglycemia_G3'
    ,'Hypoglycemia_G4'
    ,'Hypomagnesemia_G1'
    ,'Hypomagnesemia_G2'
    ,'Hypomagnesemia_G3'
    ,'Hypomagnesemia_G4'
    ,'Hyponatremia_G1'
    ,'Hyponatremia_G2'
    ,'Hyponatremia_G3'
    ,'Hyponatremia_G4'
    ,'Lymphocyte_count_decreased_G1'
    ,'Lymphocyte_count_decreased_G2'
    ,'Lymphocyte_count_decreased_G3'
    ,'Lymphocyte_count_decreased_G4'
    ,'Acidosis_arterial_G1'
    ,'Acidosis_arterial_G3']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print('base dataset:',len(IO_measurement))
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number<{}'.format(abnormal[0],abnormal[2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))

    # abnormal targetting
    c_id=abnormal[0]
    v = abnormal[2]    
    
    print('c_id:',c_id)
    print('v:',v)
    print(ab_nm)
    
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id) &\
    (IO_measurement1["value_as_number"]<v)]    
    print('\n')
print(IO_measurement_Hypoalbuminemia_G1)

In [ ]:
# 2. value 1개짜리 & increased

Hypermagnesemia_G1=[3012095,'>',1.2]
Hypermagnesemia_G3=[3012095,'>',1.23]
Hypermagnesemia_G4=[3012095,'>',3.3]
Lipase_increased_G1=[3004905,'>',60]
Lipase_increased_G2=[3004905,'>',120]
Lipase_increased_G3=[3004905,'>',300]
Serum_amylase_increased_G1=[3016771,'>',115]
Serum_amylase_increased_G2=[3016771,'>',230]
Serum_amylase_increased_G3=[3016771,'>',575]
Alanine_aminotransferase_increased_G1=[3006923,'>',46]
Alanine_aminotransferase_increased_G2=[3006923,'>',38]
Alanine_aminotransferase_increased_G3=[3006923,'>',230]
Alanine_aminotransferase_increased_G4=[3006923,'>',920]
Cholesterol_high_G1=[3027114,'>',240]
Cholesterol_high_G2=[3027114,'>',300]
Cholesterol_high_G3=[3027114,'>',400]
Cholesterol_high_G4=[3027114,'>',500]
Hyperkalemia_G1=[3023103,'>',5]
Hyperkalemia_G2=[3023103,'>',5.5]
Hyperkalemia_G3=[3023103,'>',6]
Hyperkalemia_G4=[3023103,'>',7]
Hypernatremia_G1=[3019550,'>',145]
Hypernatremia_G2=[3019550,'>',150]
Hypernatremia_G3=[3019550,'>',155]
Hypernatremia_G4=[3019550,'>',160]
Hypertriglyceridemia_G1=[3022192,'>',150]
Hypertriglyceridemia_G2=[3022192,'>',300]
Hypertriglyceridemia_G3=[3022192,'>',500]
Hypertriglyceridemia_G4=[3022192,'>',1000]
NLR_abnormal = [43055369,'>=', 3]
PLR_abnormal = [44816673,'>=', 180]
NLR_G1=[43055369,'>=',6]
NLR_G2=[43055369,'>=',9]
NLR_G3=[43055369,'>=',18]
Alkalosis_arterial_G1=[3019977,'>',7.45]
Alkalosis_arterial_G3=[3019977,'>',7.5]

abnormals=[Hypermagnesemia_G1
,Hypermagnesemia_G3
,Hypermagnesemia_G4
,Lipase_increased_G1
,Lipase_increased_G2
,Lipase_increased_G3
,Serum_amylase_increased_G1
,Serum_amylase_increased_G2
,Serum_amylase_increased_G3
,Alanine_aminotransferase_increased_G1
,Alanine_aminotransferase_increased_G2
,Alanine_aminotransferase_increased_G3
,Alanine_aminotransferase_increased_G4
,Cholesterol_high_G1
,Cholesterol_high_G2
,Cholesterol_high_G3
,Cholesterol_high_G4
,Hyperkalemia_G1
,Hyperkalemia_G2
,Hyperkalemia_G3
,Hyperkalemia_G4
,Hypernatremia_G1
,Hypernatremia_G2
,Hypernatremia_G3
,Hypernatremia_G4
,Hypertriglyceridemia_G1
,Hypertriglyceridemia_G2
,Hypertriglyceridemia_G3
,Hypertriglyceridemia_G4
,NLR_abnormal 
,PLR_abnormal 
,NLR_G1
,NLR_G2
,NLR_G3
,Alkalosis_arterial_G1
,Alkalosis_arterial_G3]

abnormals_nm=['Hypermagnesemia_G1'
,'Hypermagnesemia_G3'
,'Hypermagnesemia_G4'
,'Lipase_increased_G1'
,'Lipase_increased_G2'
,'Lipase_increased_G3'
,'Serum_amylase_increased_G1'
,'Serum_amylase_increased_G2'
,'Serum_amylase_increased_G3'
,'Alanine_aminotransferase_increased_G1'
,'Alanine_aminotransferase_increased_G2'
,'Alanine_aminotransferase_increased_G3'
,'Alanine_aminotransferase_increased_G4'
,'Cholesterol_high_G1'
,'Cholesterol_high_G2'
,'Cholesterol_high_G3'
,'Cholesterol_high_G4'
,'Hyperkalemia_G1'
,'Hyperkalemia_G2'
,'Hyperkalemia_G3'
,'Hyperkalemia_G4'
,'Hypernatremia_G1'
,'Hypernatremia_G2'
,'Hypernatremia_G3'
,'Hypernatremia_G4'
,'Hypertriglyceridemia_G1'
,'Hypertriglyceridemia_G2'
,'Hypertriglyceridemia_G3'
,'Hypertriglyceridemia_G4'
,'NLR_abnormal'
,'PLR_abnormal'
,'NLR_G1'
,'NLR_G2'
,'NLR_G3'
,'Alkalosis_arterial_G1'
,'Alkalosis_arterial_G3']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print('base dataset:',len(IO_measurement))
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{}'.format(abnormal[0],abnormal[2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))

    # abnormal targetting
    c_id=abnormal[0]
    v = abnormal[2]    
    
    print(ab_nm)
    
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id) &\
    (IO_measurement1["value_as_number"]>v)]
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
# 3. concept_id 2개이상 & value 1개 & decreased
hyperthyroidism = [[3019170,3009201],'<', 0.41]
Platelet_count_decreased_G1=[[3024929,3007461],'<',150]
Platelet_count_decreased_G2=[[3024929,3007461],'<',75]
Platelet_count_decreased_G3=[[3024929,3007461],'<',50]
Platelet_count_decreased_G4=[[3024929,3007461],'<',25]
White_blood_cell_decreased_G1=[[3010813,3000905],'<',3.78]
White_blood_cell_decreased_G2=[[3010813,3000905],'<',3]
White_blood_cell_decreased_G3=[[3010813,3000905],'<',2]
White_blood_cell_decreased_G4=[[3010813,3000905],'<',1]

# concept_id 2개 짜리
    # decreased
abnormals=[hyperthyroidism
,Platelet_count_decreased_G1
,Platelet_count_decreased_G2
,Platelet_count_decreased_G3
,Platelet_count_decreased_G4
,White_blood_cell_decreased_G1
,White_blood_cell_decreased_G2
,White_blood_cell_decreased_G3
,White_blood_cell_decreased_G4]

abnormals_nm=['hyperthyroidism'
,'Platelet_count_decreased_G1'
,'Platelet_count_decreased_G2'
,'Platelet_count_decreased_G3'
,'Platelet_count_decreased_G4'
,'White_blood_cell_decreased_G1'
,'White_blood_cell_decreased_G2'
,'White_blood_cell_decreased_G3'
,'White_blood_cell_decreased_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    print(abnormal[0])
    # 과거력 제거
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id in ({}) \
                                           and value_as_number<{}'.format(abnormal[0],abnormal[2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=abnormal[0]    
    v = abnormal[2]        
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]<v)]
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#4. concept_id 2개이상 & value 1개 & increased
hypothyroidism=[[3019170,3009201],'>',4.77]
Leukocytosis_G3=[[3010813,3000905],'>',100]
Lymphocyte_count_increased_G2=[[3004327,3019198],'>',4]
Lymphocyte_count_increased_G3=[[3004327,3019198],'>',20]
Activated_partial_thromboplastin_time_prolonged_G1=[[3018677,3038102,3013466],'>',40.6]
Activated_partial_thromboplastin_time_prolonged_G2=[[3018677,3038102,3013466],'>',60.9]
Activated_partial_thromboplastin_time_prolonged_G3=[[3018677,3038102,3013466],'>',101.5]
INR_increased_G1=[[3002417,3032080,3022217],'>',1.2]
INR_increased_G2=[[3002417,3032080,3022217],'>',1.5]
INR_increased_G3=[[3002417,3032080,3022217],'>',2.5]
Aspartate_aminotransferase_increased_G1=[[3013721,3042781],'>',34]
Aspartate_aminotransferase_increased_G2=[[3013721,3042781],'>',102]
Aspartate_aminotransferase_increased_G3=[[3013721,3042781],'>',170]
Aspartate_aminotransferase_increased_G4=[[3013721,3042781],'>',680]
Blood_lactate_dehydrogenase_increased_G1=[[3034022,3022250],'>',247]

abnormals=[hypothyroidism
,Leukocytosis_G3
,Lymphocyte_count_increased_G2
,Lymphocyte_count_increased_G3
,Activated_partial_thromboplastin_time_prolonged_G1
,Activated_partial_thromboplastin_time_prolonged_G2
,Activated_partial_thromboplastin_time_prolonged_G3
,INR_increased_G1
,INR_increased_G2
,INR_increased_G3
,Aspartate_aminotransferase_increased_G1
,Aspartate_aminotransferase_increased_G2
,Aspartate_aminotransferase_increased_G3
,Aspartate_aminotransferase_increased_G4
,Blood_lactate_dehydrogenase_increased_G1]

abnormals_nm=['hypothyroidism'
,'Leukocytosis_G3'
,'Lymphocyte_count_increased_G2'
,'Lymphocyte_count_increased_G3'
,'Activated_partial_thromboplastin_time_prolonged_G1'
,'Activated_partial_thromboplastin_time_prolonged_G2'
,'Activated_partial_thromboplastin_time_prolonged_G3'
,'INR_increased_G1'
,'INR_increased_G2'
,'INR_increased_G3'
,'Aspartate_aminotransferase_increased_G1'
,'Aspartate_aminotransferase_increased_G2'
,'Aspartate_aminotransferase_increased_G3'
,'Aspartate_aminotransferase_increased_G4'
,'Blood_lactate_dehydrogenase_increased_G1']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    print(abnormal[0])
    # 과거력 제거
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id in ({}) \
                                           and value_as_number>{}'.format(abnormal[0],abnormal[2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=abnormal[0]    
    v = abnormal[2]        
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]>v)]
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#5. concept_id 3개짜리 & value 3개 & decreased
Hypocalcemia_ca_G1=[[3006906,40762533,3036426],'<',[8.5,4.5,4.5]]
Hypocalcemia_ca_G2=[[3006906,40762533,3036426],'<',[8,4,4]]
Hypocalcemia_ca_G3=[[3006906,40762533,3036426],'<',[7,3.6,3.6]]
Hypocalcemia_ca_G4=[[3006906,40762533,3036426],'<',[6,3.2,3.2]]

abnormals=[Hypocalcemia_ca_G1
           ,Hypocalcemia_ca_G2
           ,Hypocalcemia_ca_G3
           ,Hypocalcemia_ca_G4]

abnormals_nm=['Hypocalcemia_ca_G1'
              ,'Hypocalcemia_ca_G2'
              ,'Hypocalcemia_ca_G3'
              ,'Hypocalcemia_ca_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0])
    print('base dataset:',len(IO_measurement))
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                           and value_as_number<{} or \
                                            cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                            and value_as_number<{} or \
                                            cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                            and value_as_number<{}'.format(abnormal[0][0],abnormal[2][0], \
                                                                           abnormal[0][1],abnormal[2][1], \
                                                                           abnormal[0][2],abnormal[2][2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    

    print('ab_nm:',ab_nm)
    c_id=abnormal[0][0]
    v = abnormal[2][0]
    c_id2=abnormal[0][1]
    v2 = abnormal[2][1]
    c_id3=abnormal[0][2]
    v3 = abnormal[2][2]
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id) &\
    (IO_measurement1["value_as_number"]<v) | \
    (IO_measurement1["measurement_concept_id"]==c_id2) &\
    (IO_measurement1["value_as_number"]<v2) | \
    (IO_measurement1["measurement_concept_id"]==c_id3) &\
    (IO_measurement1["value_as_number"]<v3) ]
    print('base dataset:',len(IO_measurement))    
    print('\n')
    

In [ ]:
#6. concept_id 2개짜리 & value 2개 & increased

Hyperuricemia_24hr_urie_serum = [[3021835,3037556],'>', [750 ,7] ]

abnormals=[Hyperuricemia_24hr_urie_serum]

abnormals_nm=['Hyperuricemia_24hr_urie_serum']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0])
    print('base dataset:',len(IO_measurement))
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                           and value_as_number>{} or \
                                            cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                            and value_as_number>{}'.format(abnormal[0][0],abnormal[2][0], \
                                                                           abnormal[0][1],abnormal[2][1]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    

    print('ab_nm:',ab_nm)
    c_id=abnormal[0][0]
    v = abnormal[2][0]
    c_id2=abnormal[0][1]
    v2 = abnormal[2][1]
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id) &\
    (IO_measurement1["value_as_number"]>v) | \
    (IO_measurement1["measurement_concept_id"]==c_id2) &\
    (IO_measurement1["value_as_number"]>v2) ]
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#7. concept_id 3개짜리 & value 3개 & increased

Hypercalcemia_ca_G1=[[3006906,40762533,3036426],'>',[10.8, 5.2, 5.2]]
Hypercalcemia_ca_G2=[[3006906,40762533,3036426],'>',[11.5, 6, 6]]
Hypercalcemia_ca_G3=[[3006906,40762533,3036426],'>',[12.5, 6.4, 6.4]]
Hypercalcemia_ca_G4=[[3006906,40762533,3036426],'>',[13.5, 7.2, 7.2]]

abnormals=[Hypercalcemia_ca_G1, Hypercalcemia_ca_G2, Hypercalcemia_ca_G3, Hypercalcemia_ca_G4]

abnormals_nm=['Hypercalcemia_ca_G1','Hypercalcemia_ca_G2','Hypercalcemia_ca_G3','Hypercalcemia_ca_G4',]

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0])
    print('base dataset:',len(IO_measurement))
    medical_history = IO_measurement.query(\
                                           'cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                           and value_as_number>{} or \
                                            cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                            and value_as_number>{} or \
                                            cohort_start_date>=measurement_date and measurement_concept_id =={} \
                                            and value_as_number>{}'.format(abnormal[0][0],abnormal[2][0], \
                                                                           abnormal[0][1],abnormal[2][1], \
                                                                           abnormal[0][2],abnormal[2][2]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    

    print('ab_nm:',ab_nm)
    c_id=abnormal[0][0]
    v = abnormal[2][0]
    c_id2=abnormal[0][1]
    v2 = abnormal[2][1]
    c_id3=abnormal[0][2]
    v3 = abnormal[2][2]
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id) &\
    (IO_measurement1["value_as_number"]>v) | \
    (IO_measurement1["measurement_concept_id"]==c_id2) &\
    (IO_measurement1["value_as_number"]>v2) | \
    (IO_measurement1["measurement_concept_id"]==c_id3) &\
    (IO_measurement1["value_as_number"]>v3)]
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#8. Gender & 각 concept_id/value 1개 & decreased

Anemia_sum_G1=[['F',3000963,'<',11.4],['M',3000963,'<',13]]
Anemia_sum_G2=[['F',3000963,'<',10],['M',3000963,'<',10]]
Anemia_sum_G3=[['F',3000963,'<',8],['M',3000963,'<',8]]

# gender 로 나뉘는 outcome
    #decreased

abnormals=[Anemia_sum_G1,Anemia_sum_G2,Anemia_sum_G3]
abnormals_nm=['Anemia_sum_G1','Anemia_sum_G2','Anemia_sum_G3']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0][0],abnormal[0][1],abnormal[0][3])
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           """gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number<{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number<{}""".format(abnormal[0][0],abnormal[0][1],abnormal[0][3], \
                                                                            abnormal[1][0],abnormal[1][1],abnormal[1][3]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=abnormal[0][1]    
    v  = abnormal[0][3]        
    v2 = abnormal[1][3]        
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]<v)\
     |\
     (IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]<v2) ]
    
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#9. Gender & 각 concept_id/value 1개 & increased

CPK_increased_sum_G1=[['F','M'],[3015620,3007220],'>',[135,245]] #cocept_id는 합집합
CPK_increased_sum_G2=[['F','M'],[3015620,3007220],'>',[337.5,612.5]] #cocept_id는 합집합
CPK_increased_sum_G3=[['F','M'],[3015620,3007220],'>',[675,1225]] #cocept_id는 합집합
CPK_increased_sum_G4=[['F','M'],[3015620,3007220],'>',[1350,2450]] #cocept_id는 합집합

abnormals=[CPK_increased_sum_G1,CPK_increased_sum_G2,CPK_increased_sum_G3,CPK_increased_sum_G4]
abnormals_nm=['CPK_increased_sum_G1','CPK_increased_sum_G2','CPK_increased_sum_G3','CPK_increased_sum_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0][0],abnormal[0][1],abnormal[3][0])
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           """gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id in ({},{}) \
                                           and value_as_number>{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id in ({},{}) \
                                           and value_as_number>{}""".format(abnormal[0][0],abnormal[1][0],abnormal[1][1],abnormal[3][0], \
                                                                            abnormal[0][1],abnormal[1][0],abnormal[1][1],abnormal[3][1]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=[abnormal[1][0],abnormal[1][1]]
    v  = abnormal[3][0]       
    v2 = abnormal[3][1]     
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]>v)\
     |\
     (IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]>v2) ]
    
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
# 10. Gender & 동일 concept_id & 각 value 1개 & decreased

Neutrophil_count_decreased_G1=[['F','M'],[3013650,3017732],'<',[1.6,1.7]]
Neutrophil_count_decreased_G2=[['F','M'],[3013650,3017732],'<',[1.5,1.5]]
Neutrophil_count_decreased_G3=[['F','M'],[3013650,3017732],'<',[1.0,1.0]]
Neutrophil_count_decreased_G4=[['F','M'],[3013650,3017732],'<',[0.5,0.5]]

abnormals=[Neutrophil_count_decreased_G1,Neutrophil_count_decreased_G2,Neutrophil_count_decreased_G3,Neutrophil_count_decreased_G4]
abnormals_nm=['Neutrophil_count_decreased_G1','Neutrophil_count_decreased_G2','Neutrophil_count_decreased_G3','Neutrophil_count_decreased_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0][0],abnormal[0][1],abnormal[3][0])
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           """gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id in ({},{}) \
                                           and value_as_number<{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id in ({},{}) \
                                           and value_as_number<{}""".format(abnormal[0][0],abnormal[1][0],abnormal[1][1],abnormal[3][0], \
                                                                            abnormal[0][1],abnormal[1][0],abnormal[1][1],abnormal[3][1]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=[abnormal[1][0],abnormal[1][1]]
    v  = abnormal[3][0]       
    v2 = abnormal[3][1]     
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]<v)\
     |\
     (IO_measurement1["measurement_concept_id"].isin(c_id) )  &\
    (IO_measurement1["value_as_number"]<v2) ]
    
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#11. Gender & 동일 concept_id & 각 value 1개 & increased

Alkaline_phosphatase_increased_G1=[['F','M'],3035995,'>',[390,151]]
Alkaline_phosphatase_increased_G2=[['F','M'],3035995,'>',[975,377.5]]
Alkaline_phosphatase_increased_G3=[['F','M'],3035995,'>',[1950,775]]
Alkaline_phosphatase_increased_G4=[['F','M'],3035995,'>',[7800,3020]]
Hemoglobin_increased_sum_G1=[['F','M'],3000963,'>',[16,18]]
Hemoglobin_increased_sum_G2=[['F','M'],3000963,'>',[18,20]]
Hemoglobin_increased_sum_G3=[['F','M'],3000963,'>',[20,22]]
creatinine_increased_sum_G1=[['F','M'],3016723,'>',[0.91,1.19]]
creatinine_increased_sum_G2=[['F','M'],3016723,'>',[1.365,1.785]]
creatinine_increased_sum_G3=[['F','M'],3016723,'>',[2.73,3.57]]
creatinine_increased_sum_G4=[['F','M'],3016723,'>',[5.46,7.14]]
GGT_increased_sum_G1=[['F','M'],3026910,'>',[54,54]]
GGT_increased_sum_G2=[['F','M'],3026910,'>',[135,135]]
GGT_increased_sum_G3=[['F','M'],3026910,'>',[270,270]]
GGT_increased_sum_G4=[['F','M'],3026910,'>',[1080,1080]]

abnormals=[Alkaline_phosphatase_increased_G1
,Alkaline_phosphatase_increased_G2
,Alkaline_phosphatase_increased_G3
,Alkaline_phosphatase_increased_G4
,Hemoglobin_increased_sum_G1
,Hemoglobin_increased_sum_G2
,Hemoglobin_increased_sum_G3
,creatinine_increased_sum_G1
,creatinine_increased_sum_G2
,creatinine_increased_sum_G3
,creatinine_increased_sum_G4
,GGT_increased_sum_G1
,GGT_increased_sum_G2
,GGT_increased_sum_G3
,GGT_increased_sum_G4]

abnormals_nm=['Alkaline_phosphatase_increased_G1'
,'Alkaline_phosphatase_increased_G2'
,'Alkaline_phosphatase_increased_G3'
,'Alkaline_phosphatase_increased_G4'
,'Hemoglobin_increased_sum_G1'
,'Hemoglobin_increased_sum_G2'
,'Hemoglobin_increased_sum_G3'
,'creatinine_increased_sum_G1'
,'creatinine_increased_sum_G2'
,'creatinine_increased_sum_G3'
,'creatinine_increased_sum_G4'
,'GGT_increased_sum_G1'
,'GGT_increased_sum_G2'
,'GGT_increased_sum_G3'
,'GGT_increased_sum_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0][0],abnormal[0][1],abnormal[1])
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           """gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{}""".format(abnormal[0][0],abnormal[1],abnormal[3][0], \
                                                                            abnormal[0][1],abnormal[1],abnormal[3][1]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=abnormal[1]    
    v  = abnormal[3][0]        
    v2 = abnormal[3][1]        
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]>v)\
     |\
     (IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]>v2) ]
    
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
#12. Gender & 동일 concept_id & 각 value 1개 & increased

Blood_bilirubin_increased_G1=[['F','M'],[3024128,3028833],'>',[1.5,1.8]]
Blood_bilirubin_increased_G2=[['F','M'],[3024128,3028833],'>',[2.25,2.7]]
Blood_bilirubin_increased_G3=[['F','M'],[3024128,3028833],'>',[4.5,5.4]]
Blood_bilirubin_increased_G4=[['F','M'],[3024128,3028833],'>',[15,18]]

abnormals=[Blood_bilirubin_increased_G1
,Blood_bilirubin_increased_G2
,Blood_bilirubin_increased_G3
,Blood_bilirubin_increased_G4]

abnormals_nm=['Blood_bilirubin_increased_G1'
,'Blood_bilirubin_increased_G2'
,'Blood_bilirubin_increased_G3'
,'Blood_bilirubin_increased_G4']

for abnormal, ab_nm in zip(abnormals,abnormals_nm):
    # 과거력 제거
    print(abnormal[0][0],abnormal[0][1],abnormal[1])
    print('base dataset:',len(IO_measurement))    
    medical_history = IO_measurement.query(\
                                           """gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{} \
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{}\
                                           or \
                                           gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} \
                                           and value_as_number>{}""".format(abnormal[0][0],abnormal[1][0],abnormal[3][0], \
                                                                            abnormal[0][1],abnormal[1][0],abnormal[3][1], \
                                                                            abnormal[0][1],abnormal[1][1],abnormal[3][1], \
                                                                            abnormal[0][1],abnormal[1][1],abnormal[3][1]) )    
    medical_history_person = medical_history.subject_id.unique()                
    IO_measurement1 = IO_measurement[~IO_measurement["subject_id"].isin(medical_history_person)]
    globals()['IO_measurement_{}'.format(ab_nm)] = IO_measurement1
    print('after wash out:', len(IO_measurement1))  
    
    print(ab_nm)
    # abnormal targetting_hypo
    c_id=abnormal[1][0]        
    c_id2=abnormal[1][1]    
    v  = abnormal[3][0]        
    v2 = abnormal[3][1]        
    globals()['abnormal_{}'.format(ab_nm)] = IO_measurement1\
    [(IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]>v)\
     |\
     (IO_measurement1["measurement_concept_id"]==c_id )  &\
    (IO_measurement1["value_as_number"]>v2)\
     |\
     (IO_measurement1["measurement_concept_id"]==c_id2 )  &\
    (IO_measurement1["value_as_number"]>v)\
     |\
     (IO_measurement1["measurement_concept_id"]==c_id2 )  &\
    (IO_measurement1["value_as_number"]>v2) ]
    
    print('base dataset:',len(IO_measurement))    
    print('\n')

In [ ]:
IO_measurement_abnormal=[IO_measurement_Hypoalbuminemia_G1
,IO_measurement_Hypoalbuminemia_G2
,IO_measurement_Hypoalbuminemia_G3
,IO_measurement_Hypokalemia_G1
,IO_measurement_Hypokalemia_G3
,IO_measurement_Hypokalemia_G4
,IO_measurement_Fibrinogen_decreased_G1
,IO_measurement_Fibrinogen_decreased_G2
,IO_measurement_Fibrinogen_decreased_G3
,IO_measurement_Fibrinogen_decreased_G4
,IO_measurement_Hypoglycemia_G1
,IO_measurement_Hypoglycemia_G2
,IO_measurement_Hypoglycemia_G3
,IO_measurement_Hypoglycemia_G4
,IO_measurement_Hypomagnesemia_G1
,IO_measurement_Hypomagnesemia_G2
,IO_measurement_Hypomagnesemia_G3
,IO_measurement_Hypomagnesemia_G4
,IO_measurement_Hyponatremia_G1
,IO_measurement_Hyponatremia_G2
,IO_measurement_Hyponatremia_G3
,IO_measurement_Hyponatremia_G4
,IO_measurement_Lymphocyte_count_decreased_G1
,IO_measurement_Lymphocyte_count_decreased_G2
,IO_measurement_Lymphocyte_count_decreased_G3
,IO_measurement_Lymphocyte_count_decreased_G4
,IO_measurement_Acidosis_arterial_G1
,IO_measurement_Acidosis_arterial_G3
,IO_measurement_Hypermagnesemia_G1
,IO_measurement_Hypermagnesemia_G3
,IO_measurement_Hypermagnesemia_G4
,IO_measurement_Lipase_increased_G1
,IO_measurement_Lipase_increased_G2
,IO_measurement_Lipase_increased_G3
,IO_measurement_Serum_amylase_increased_G1
,IO_measurement_Serum_amylase_increased_G2
,IO_measurement_Serum_amylase_increased_G3
,IO_measurement_Alanine_aminotransferase_increased_G1
,IO_measurement_Alanine_aminotransferase_increased_G2
,IO_measurement_Alanine_aminotransferase_increased_G3
,IO_measurement_Alanine_aminotransferase_increased_G4
,IO_measurement_Cholesterol_high_G1
,IO_measurement_Cholesterol_high_G2
,IO_measurement_Cholesterol_high_G3
,IO_measurement_Cholesterol_high_G4
,IO_measurement_Hyperkalemia_G1
,IO_measurement_Hyperkalemia_G2
,IO_measurement_Hyperkalemia_G3
,IO_measurement_Hyperkalemia_G4
,IO_measurement_Hypernatremia_G1
,IO_measurement_Hypernatremia_G2
,IO_measurement_Hypernatremia_G3
,IO_measurement_Hypernatremia_G4
,IO_measurement_Hypertriglyceridemia_G1
,IO_measurement_Hypertriglyceridemia_G2
,IO_measurement_Hypertriglyceridemia_G3
,IO_measurement_Hypertriglyceridemia_G4
,IO_measurement_NLR_abnormal 
,IO_measurement_PLR_abnormal 
,IO_measurement_NLR_G1
,IO_measurement_NLR_G2
,IO_measurement_NLR_G3
,IO_measurement_Alkalosis_arterial_G1
,IO_measurement_Alkalosis_arterial_G3
,IO_measurement_hyperthyroidism 
,IO_measurement_Platelet_count_decreased_G1
,IO_measurement_Platelet_count_decreased_G2
,IO_measurement_Platelet_count_decreased_G3
,IO_measurement_Platelet_count_decreased_G4
,IO_measurement_White_blood_cell_decreased_G1
,IO_measurement_White_blood_cell_decreased_G2
,IO_measurement_White_blood_cell_decreased_G3
,IO_measurement_White_blood_cell_decreased_G4
,IO_measurement_hypothyroidism
,IO_measurement_Leukocytosis_G3
,IO_measurement_Lymphocyte_count_increased_G2
,IO_measurement_Lymphocyte_count_increased_G3
,IO_measurement_Activated_partial_thromboplastin_time_prolonged_G1
,IO_measurement_Activated_partial_thromboplastin_time_prolonged_G2
,IO_measurement_Activated_partial_thromboplastin_time_prolonged_G3
,IO_measurement_INR_increased_G1
,IO_measurement_INR_increased_G2
,IO_measurement_INR_increased_G3
,IO_measurement_Aspartate_aminotransferase_increased_G1
,IO_measurement_Aspartate_aminotransferase_increased_G2
,IO_measurement_Aspartate_aminotransferase_increased_G3
,IO_measurement_Aspartate_aminotransferase_increased_G4
,IO_measurement_Blood_lactate_dehydrogenase_increased_G1
,IO_measurement_Hypocalcemia_ca_G1
,IO_measurement_Hypocalcemia_ca_G2
,IO_measurement_Hypocalcemia_ca_G3
,IO_measurement_Hypocalcemia_ca_G4
,IO_measurement_Hyperuricemia_24hr_urie_serum 
,IO_measurement_Hypercalcemia_ca_G1
,IO_measurement_Hypercalcemia_ca_G2
,IO_measurement_Hypercalcemia_ca_G3
,IO_measurement_Hypercalcemia_ca_G4
,IO_measurement_Anemia_sum_G1
,IO_measurement_Anemia_sum_G2
,IO_measurement_Anemia_sum_G3
,IO_measurement_CPK_increased_sum_G1
,IO_measurement_CPK_increased_sum_G2
,IO_measurement_CPK_increased_sum_G3
,IO_measurement_CPK_increased_sum_G4
,IO_measurement_Neutrophil_count_decreased_G1
,IO_measurement_Neutrophil_count_decreased_G2
,IO_measurement_Neutrophil_count_decreased_G3
,IO_measurement_Neutrophil_count_decreased_G4
,IO_measurement_Alkaline_phosphatase_increased_G1
,IO_measurement_Alkaline_phosphatase_increased_G2
,IO_measurement_Alkaline_phosphatase_increased_G3
,IO_measurement_Alkaline_phosphatase_increased_G4
,IO_measurement_Hemoglobin_increased_sum_G1
,IO_measurement_Hemoglobin_increased_sum_G2
,IO_measurement_Hemoglobin_increased_sum_G3
,IO_measurement_creatinine_increased_sum_G1
,IO_measurement_creatinine_increased_sum_G2
,IO_measurement_creatinine_increased_sum_G3
,IO_measurement_creatinine_increased_sum_G4
,IO_measurement_GGT_increased_sum_G1
,IO_measurement_GGT_increased_sum_G2
,IO_measurement_GGT_increased_sum_G3
,IO_measurement_GGT_increased_sum_G4
,IO_measurement_Blood_bilirubin_increased_G1
,IO_measurement_Blood_bilirubin_increased_G2
,IO_measurement_Blood_bilirubin_increased_G3
,IO_measurement_Blood_bilirubin_increased_G4]

abnormal_list=[abnormal_Hypoalbuminemia_G1
,abnormal_Hypoalbuminemia_G2
,abnormal_Hypoalbuminemia_G3
,abnormal_Hypokalemia_G1
,abnormal_Hypokalemia_G3
,abnormal_Hypokalemia_G4
,abnormal_Fibrinogen_decreased_G1
,abnormal_Fibrinogen_decreased_G2
,abnormal_Fibrinogen_decreased_G3
,abnormal_Fibrinogen_decreased_G4
,abnormal_Hypoglycemia_G1
,abnormal_Hypoglycemia_G2
,abnormal_Hypoglycemia_G3
,abnormal_Hypoglycemia_G4
,abnormal_Hypomagnesemia_G1
,abnormal_Hypomagnesemia_G2
,abnormal_Hypomagnesemia_G3
,abnormal_Hypomagnesemia_G4
,abnormal_Hyponatremia_G1
,abnormal_Hyponatremia_G2
,abnormal_Hyponatremia_G3
,abnormal_Hyponatremia_G4
,abnormal_Lymphocyte_count_decreased_G1
,abnormal_Lymphocyte_count_decreased_G2
,abnormal_Lymphocyte_count_decreased_G3
,abnormal_Lymphocyte_count_decreased_G4
,abnormal_Acidosis_arterial_G1
,abnormal_Acidosis_arterial_G3
,abnormal_Hypermagnesemia_G1
,abnormal_Hypermagnesemia_G3
,abnormal_Hypermagnesemia_G4
,abnormal_Lipase_increased_G1
,abnormal_Lipase_increased_G2
,abnormal_Lipase_increased_G3
,abnormal_Serum_amylase_increased_G1
,abnormal_Serum_amylase_increased_G2
,abnormal_Serum_amylase_increased_G3
,abnormal_Alanine_aminotransferase_increased_G1
,abnormal_Alanine_aminotransferase_increased_G2
,abnormal_Alanine_aminotransferase_increased_G3
,abnormal_Alanine_aminotransferase_increased_G4
,abnormal_Cholesterol_high_G1
,abnormal_Cholesterol_high_G2
,abnormal_Cholesterol_high_G3
,abnormal_Cholesterol_high_G4
,abnormal_Hyperkalemia_G1
,abnormal_Hyperkalemia_G2
,abnormal_Hyperkalemia_G3
,abnormal_Hyperkalemia_G4
,abnormal_Hypernatremia_G1
,abnormal_Hypernatremia_G2
,abnormal_Hypernatremia_G3
,abnormal_Hypernatremia_G4
,abnormal_Hypertriglyceridemia_G1
,abnormal_Hypertriglyceridemia_G2
,abnormal_Hypertriglyceridemia_G3
,abnormal_Hypertriglyceridemia_G4
,abnormal_NLR_abnormal 
,abnormal_PLR_abnormal 
,abnormal_NLR_G1
,abnormal_NLR_G2
,abnormal_NLR_G3
,abnormal_Alkalosis_arterial_G1
,abnormal_Alkalosis_arterial_G3
,abnormal_hyperthyroidism 
,abnormal_Platelet_count_decreased_G1
,abnormal_Platelet_count_decreased_G2
,abnormal_Platelet_count_decreased_G3
,abnormal_Platelet_count_decreased_G4
,abnormal_White_blood_cell_decreased_G1
,abnormal_White_blood_cell_decreased_G2
,abnormal_White_blood_cell_decreased_G3
,abnormal_White_blood_cell_decreased_G4
,abnormal_hypothyroidism
,abnormal_Leukocytosis_G3
,abnormal_Lymphocyte_count_increased_G2
,abnormal_Lymphocyte_count_increased_G3
,abnormal_Activated_partial_thromboplastin_time_prolonged_G1
,abnormal_Activated_partial_thromboplastin_time_prolonged_G2
,abnormal_Activated_partial_thromboplastin_time_prolonged_G3
,abnormal_INR_increased_G1
,abnormal_INR_increased_G2
,abnormal_INR_increased_G3
,abnormal_Aspartate_aminotransferase_increased_G1
,abnormal_Aspartate_aminotransferase_increased_G2
,abnormal_Aspartate_aminotransferase_increased_G3
,abnormal_Aspartate_aminotransferase_increased_G4
,abnormal_Blood_lactate_dehydrogenase_increased_G1
,abnormal_Hypocalcemia_ca_G1
,abnormal_Hypocalcemia_ca_G2
,abnormal_Hypocalcemia_ca_G3
,abnormal_Hypocalcemia_ca_G4
,abnormal_Hyperuricemia_24hr_urie_serum 
,abnormal_Hypercalcemia_ca_G1
,abnormal_Hypercalcemia_ca_G2
,abnormal_Hypercalcemia_ca_G3
,abnormal_Hypercalcemia_ca_G4
,abnormal_Anemia_sum_G1
,abnormal_Anemia_sum_G2
,abnormal_Anemia_sum_G3
,abnormal_CPK_increased_sum_G1
,abnormal_CPK_increased_sum_G2
,abnormal_CPK_increased_sum_G3
,abnormal_CPK_increased_sum_G4
,abnormal_Neutrophil_count_decreased_G1
,abnormal_Neutrophil_count_decreased_G2
,abnormal_Neutrophil_count_decreased_G3
,abnormal_Neutrophil_count_decreased_G4
,abnormal_Alkaline_phosphatase_increased_G1
,abnormal_Alkaline_phosphatase_increased_G2
,abnormal_Alkaline_phosphatase_increased_G3
,abnormal_Alkaline_phosphatase_increased_G4
,abnormal_Hemoglobin_increased_sum_G1
,abnormal_Hemoglobin_increased_sum_G2
,abnormal_Hemoglobin_increased_sum_G3
,abnormal_creatinine_increased_sum_G1
,abnormal_creatinine_increased_sum_G2
,abnormal_creatinine_increased_sum_G3
,abnormal_creatinine_increased_sum_G4
,abnormal_GGT_increased_sum_G1
,abnormal_GGT_increased_sum_G2
,abnormal_GGT_increased_sum_G3
,abnormal_GGT_increased_sum_G4
,abnormal_Blood_bilirubin_increased_G1
,abnormal_Blood_bilirubin_increased_G2
,abnormal_Blood_bilirubin_increased_G3
,abnormal_Blood_bilirubin_increased_G4]


In [ ]:

abnormal_list_nm=['abnormal_Hypoalbuminemia_G1'
,'abnormal_Hypoalbuminemia_G2'
,'abnormal_Hypoalbuminemia_G3'
,'abnormal_Hypokalemia_G1'
,'abnormal_Hypokalemia_G3'
,'abnormal_Hypokalemia_G4'
,'abnormal_Fibrinogen_decreased_G1'
,'abnormal_Fibrinogen_decreased_G2'
,'abnormal_Fibrinogen_decreased_G3'
,'abnormal_Fibrinogen_decreased_G4'
,'abnormal_Hypoglycemia_G1'
,'abnormal_Hypoglycemia_G2'
,'abnormal_Hypoglycemia_G3'
,'abnormal_Hypoglycemia_G4'
,'abnormal_Hypomagnesemia_G1'
,'abnormal_Hypomagnesemia_G2'
,'abnormal_Hypomagnesemia_G3'
,'abnormal_Hypomagnesemia_G4'
,'abnormal_Hyponatremia_G1'
,'abnormal_Hyponatremia_G2'
,'abnormal_Hyponatremia_G3'
,'abnormal_Hyponatremia_G4'
,'abnormal_Lymphocyte_count_decreased_G1'
,'abnormal_Lymphocyte_count_decreased_G2'
,'abnormal_Lymphocyte_count_decreased_G3'
,'abnormal_Lymphocyte_count_decreased_G4'
,'abnormal_Acidosis_arterial_G1'
,'abnormal_Acidosis_arterial_G3'
,'abnormal_Hypermagnesemia_G1'
,'abnormal_Hypermagnesemia_G3'
,'abnormal_Hypermagnesemia_G4'
,'abnormal_Lipase_increased_G1'
,'abnormal_Lipase_increased_G2'
,'abnormal_Lipase_increased_G3'
,'abnormal_Serum_amylase_increased_G1'
,'abnormal_Serum_amylase_increased_G2'
,'abnormal_Serum_amylase_increased_G3'
,'abnormal_Alanine_aminotransferase_increased_G1'
,'abnormal_Alanine_aminotransferase_increased_G2'
,'abnormal_Alanine_aminotransferase_increased_G3'
,'abnormal_Alanine_aminotransferase_increased_G4'
,'abnormal_Cholesterol_high_G1'
,'abnormal_Cholesterol_high_G2'
,'abnormal_Cholesterol_high_G3'
,'abnormal_Cholesterol_high_G4'
,'abnormal_Hyperkalemia_G1'
,'abnormal_Hyperkalemia_G2'
,'abnormal_Hyperkalemia_G3'
,'abnormal_Hyperkalemia_G4'
,'abnormal_Hypernatremia_G1'
,'abnormal_Hypernatremia_G2'
,'abnormal_Hypernatremia_G3'
,'abnormal_Hypernatremia_G4'
,'abnormal_Hypertriglyceridemia_G1'
,'abnormal_Hypertriglyceridemia_G2'
,'abnormal_Hypertriglyceridemia_G3'
,'abnormal_Hypertriglyceridemia_G4'
,'abnormal_NLR_abnormal'
,'abnormal_PLR_abnormal'
,'abnormal_NLR_G1'
,'abnormal_NLR_G2'
,'abnormal_NLR_G3'
,'abnormal_Alkalosis_arterial_G1'
,'abnormal_Alkalosis_arterial_G3'
,'abnormal_hyperthyroidism'
,'abnormal_Platelet_count_decreased_G1'
,'abnormal_Platelet_count_decreased_G2'
,'abnormal_Platelet_count_decreased_G3'
,'abnormal_Platelet_count_decreased_G4'
,'abnormal_White_blood_cell_decreased_G1'
,'abnormal_White_blood_cell_decreased_G2'
,'abnormal_White_blood_cell_decreased_G3'
,'abnormal_White_blood_cell_decreased_G4'
,'abnormal_hypothyroidism'
,'abnormal_Leukocytosis_G3'
,'abnormal_Lymphocyte_count_increased_G2'
,'abnormal_Lymphocyte_count_increased_G3'
,'abnormal_Activated_partial_thromboplastin_time_prolonged_G1'
,'abnormal_Activated_partial_thromboplastin_time_prolonged_G2'
,'abnormal_Activated_partial_thromboplastin_time_prolonged_G3'
,'abnormal_INR_increased_G1'
,'abnormal_INR_increased_G2'
,'abnormal_INR_increased_G3'
,'abnormal_Aspartate_aminotransferase_increased_G1'
,'abnormal_Aspartate_aminotransferase_increased_G2'
,'abnormal_Aspartate_aminotransferase_increased_G3'
,'abnormal_Aspartate_aminotransferase_increased_G4'
,'abnormal_Blood_lactate_dehydrogenase_increased_G1'
,'abnormal_Hypocalcemia_ca_G1'
,'abnormal_Hypocalcemia_ca_G2'
,'abnormal_Hypocalcemia_ca_G3'
,'abnormal_Hypocalcemia_ca_G4'
,'abnormal_Hyperuricemia_24hr_urie_serum'
,'abnormal_Hypercalcemia_ca_G1'
,'abnormal_Hypercalcemia_ca_G2'
,'abnormal_Hypercalcemia_ca_G3'
,'abnormal_Hypercalcemia_ca_G4'
,'abnormal_Anemia_sum_G1'
,'abnormal_Anemia_sum_G2'
,'abnormal_Anemia_sum_G3'
,'abnormal_CPK_increased_sum_G1'
,'abnormal_CPK_increased_sum_G2'
,'abnormal_CPK_increased_sum_G3'
,'abnormal_CPK_increased_sum_G4'
,'abnormal_Neutrophil_count_decreased_G1'
,'abnormal_Neutrophil_count_decreased_G2'
,'abnormal_Neutrophil_count_decreased_G3'
,'abnormal_Neutrophil_count_decreased_G4'
,'abnormal_Alkaline_phosphatase_increased_G1'
,'abnormal_Alkaline_phosphatase_increased_G2'
,'abnormal_Alkaline_phosphatase_increased_G3'
,'abnormal_Alkaline_phosphatase_increased_G4'
,'abnormal_Hemoglobin_increased_sum_G1'
,'abnormal_Hemoglobin_increased_sum_G2'
,'abnormal_Hemoglobin_increased_sum_G3'
,'abnormal_creatinine_increased_sum_G1'
,'abnormal_creatinine_increased_sum_G2'
,'abnormal_creatinine_increased_sum_G3'
,'abnormal_creatinine_increased_sum_G4'
,'abnormal_GGT_increased_sum_G1'
,'abnormal_GGT_increased_sum_G2'
,'abnormal_GGT_increased_sum_G3'
,'abnormal_GGT_increased_sum_G4'
,'abnormal_Blood_bilirubin_increased_G1'
,'abnormal_Blood_bilirubin_increased_G2'
,'abnormal_Blood_bilirubin_increased_G3'
,'abnormal_Blood_bilirubin_increased_G4']

In [ ]:
import pickle
filePath = './abnormal_list.txt'
with open(filePath, 'wb') as lf:
    pickle.dump(abnormal_list_nm, lf)

In [ ]:
with open('abnormal_list.txt', 'r') as f:
    data = f.read()
file_list = data.splitlines()


# outcome 마다 각 domain dataset 생성

In [ ]:
# outcome 마다 각 domain dataset 생성
domains=['_measurement','_drug','_procedure','_condition',]

#measurement
for IO_measurement, abnormal_ctcae, nm in zip(IO_measurement_abnormal, abnormal_list, abnormal_list_nm):
    dataset = measurement_targetting(abnormal_ctcae,IO_measurement)    
    
    # population
    targetting = dataset[["subject_id","cohort_start_date","age","gender_source_value","first_abnormal_date","label"]]
    targetting=targetting.drop_duplicates()
    
    #measurement
    meas_1=dataset
    meas_1.rename(columns={'gender_source_value':'sex'},inplace=True)
#    globals()['{}{}'.format(nm,domains[0])] = meas_1
    meas_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[0]),index=False)
    
    #drug    
    drug_1=pd.merge(targetting, IO_drug, left_on=["subject_id"],right_on=["person_id"],how="inner")
    drug_1=drug_1.query('cohort_start_date_x <= drug_exposure_start_date')    
    drug_1=drug_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#    globals()['{}{}'.format(nm,domains[1])] = drug_1.reset_index(drop=True)
    drug_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[1]),index=False)
    
    #procedure    
    proc_1=pd.merge(targetting, IO_procedure, left_on=["subject_id"],right_on=["person_id"],how="inner")
    proc_1=proc_1.query('cohort_start_date_x <= procedure_date')    
    proc_1=proc_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#    globals()['{}{}'.format(nm,domains[2])] = proc_1.reset_index(drop=True)
    proc_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[2]),index=False)
    
    #condition    
    cond_1=pd.merge(targetting, IO_condition, left_on=["subject_id"],right_on=["person_id"],how="inner")
    cond_1=cond_1.query('cohort_start_date_x <= condition_start_date')    
    cond_1=cond_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#   globals()['{}{}'.format(nm,domains[3])] = cond_1.reset_index(drop=True)  
    cond_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[3]),index=False) 


In [ ]:
# check
IO_lab=abnormal_Blood_bilirubin_increased_G1_drug
IO_lab

In [ ]:
# 변수확인
IO_lab[IO_lab['subject_id']==4795237].concept_name.unique()

In [ ]:
#label 갯수 확인
print("abnormal 환자(=label 1):",len(IO_lab[IO_lab["label"]==1].person_id.unique()) )
print("normal 환자(label 0):",len(IO_lab[IO_lab["label"]==0].person_id.unique()) )

# save csv

# save csv path

In [ ]:
# outcome_condition
for i,nm in zip(outcome_condition,outcome_condition_nm):    
    i.to_csv('{}/{}.txt'.format(output_dir,nm),index=False)